In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import shutil
import glob
from tqdm import tqdm

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
batch_size = 8

In [4]:
train_data_generator = ImageDataGenerator(rescale=1./255, 
                                          rotation_range=0.2,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          validation_split=0.2
                                         )

In [5]:
train_data = train_data_generator.flow_from_directory(r'PreparedData/train',
                                                      target_size=(80,80), 
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      subset='training'
                                                     )


validation_data = train_data_generator.flow_from_directory(r'PreparedData/train',
                                                      target_size=(80,80), 
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      subset='validation'
                                                     )

Found 64340 images belonging to 2 classes.
Found 16084 images belonging to 2 classes.


In [6]:
test_data_generator = ImageDataGenerator(rescale=1./255)

test_data = test_data_generator.flow_from_directory(r'PreparedData/test',
                                                    target_size=(80,80),
                                                    batch_size=batch_size,
                                                    class_mode='categorical'
                                
                                                   )

Found 4474 images belonging to 2 classes.


In [12]:
base_model = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))

In [13]:
head_model = base_model.output
head_model = Flatten()(head_model)
head_model = Dense(64, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(1,activation= 'softmax')(head_model)

model = Model(inputs=base_model.input, outputs= head_model)
for layer in base_model.layers:
    layer.trainable = False

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (None, 39, 39, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 39, 39, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (None, 39, 39, 32)   0           ['batch_normalization_94[0][0

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_105 (Batch  (None, 7, 7, 32)    96          ['conv2d_105[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_99 (Activation)     (None, 7, 7, 64)     0           ['batch_normalization_99[0][0]'] 
                                                                                                  
 activation_101 (Activation)    (None, 7, 7, 64)     0           ['batch_normalization_101[0][0]']
                                                                                                  
 activation_104 (Activation)    (None, 7, 7, 96)     0           ['batch_normalization_104[0][0]']
          

                                                                                                  
 conv2d_117 (Conv2D)            (None, 7, 7, 96)     55296       ['activation_116[0][0]']         
                                                                                                  
 batch_normalization_114 (Batch  (None, 7, 7, 48)    144         ['conv2d_114[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_117 (Batch  (None, 7, 7, 96)    288         ['conv2d_117[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_114 (Activation)    (None, 7, 7, 48)     0           ['batch_normalization_114[0][0]']
          

 conv2d_128 (Conv2D)            (None, 3, 3, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_128 (Batch  (None, 3, 3, 128)   384         ['conv2d_128[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_128 (Activation)    (None, 3, 3, 128)    0           ['batch_normalization_128[0][0]']
                                                                                                  
 conv2d_129 (Conv2D)            (None, 3, 3, 128)    114688      ['activation_128[0][0]']         
                                                                                                  
 batch_normalization_129 (Batch  (None, 3, 3, 128)   384         ['conv2d_129[0][0]']             
 Normaliza

 activation_138 (Activation)    (None, 3, 3, 160)    0           ['batch_normalization_138[0][0]']
                                                                                                  
 conv2d_139 (Conv2D)            (None, 3, 3, 160)    179200      ['activation_138[0][0]']         
                                                                                                  
 batch_normalization_139 (Batch  (None, 3, 3, 160)   480         ['conv2d_139[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_139 (Activation)    (None, 3, 3, 160)    0           ['batch_normalization_139[0][0]']
                                                                                                  
 conv2d_135 (Conv2D)            (None, 3, 3, 160)    122880      ['mixed4[0][0]']                 
          

 Normalization)                                                                                   
                                                                                                  
 activation_149 (Activation)    (None, 3, 3, 160)    0           ['batch_normalization_149[0][0]']
                                                                                                  
 conv2d_145 (Conv2D)            (None, 3, 3, 160)    122880      ['mixed5[0][0]']                 
                                                                                                  
 conv2d_150 (Conv2D)            (None, 3, 3, 160)    179200      ['activation_149[0][0]']         
                                                                                                  
 batch_normalization_145 (Batch  (None, 3, 3, 160)   480         ['conv2d_145[0][0]']             
 Normalization)                                                                                   
          

                                                                                                  
 conv2d_160 (Conv2D)            (None, 3, 3, 192)    258048      ['activation_159[0][0]']         
                                                                                                  
 batch_normalization_155 (Batch  (None, 3, 3, 192)   576         ['conv2d_155[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_160 (Batch  (None, 3, 3, 192)   576         ['conv2d_160[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_155 (Activation)    (None, 3, 3, 192)    0           ['batch_normalization_155[0][0]']
          

                                                                                                  
 batch_normalization_168 (Batch  (None, 3, 3, 192)   576         ['conv2d_168[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_164 (Activation)    (None, 3, 3, 192)    0           ['batch_normalization_164[0][0]']
                                                                                                  
 activation_168 (Activation)    (None, 3, 3, 192)    0           ['batch_normalization_168[0][0]']
                                                                                                  
 conv2d_165 (Conv2D)            (None, 1, 1, 320)    552960      ['activation_164[0][0]']         
                                                                                                  
 conv2d_16

 activation_176 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_176[0][0]']
                                                                                                  
 activation_177 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_177[0][0]']
                                                                                                  
 batch_normalization_178 (Batch  (None, 1, 1, 192)   576         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_170 (Activation)    (None, 1, 1, 320)    0           ['batch_normalization_170[0][0]']
                                                                                                  
 mixed9_0 (Concatenate)         (None, 1, 1, 768)    0           ['activation_172[0][0]',         
          

                                                                                                  
 activation_179 (Activation)    (None, 1, 1, 320)    0           ['batch_normalization_179[0][0]']
                                                                                                  
 mixed9_1 (Concatenate)         (None, 1, 1, 768)    0           ['activation_181[0][0]',         
                                                                  'activation_182[0][0]']         
                                                                                                  
 concatenate_3 (Concatenate)    (None, 1, 1, 768)    0           ['activation_185[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 activation_187 (Activation)    (None, 1, 1, 192)    0           ['batch_normalization_187[0][0]']
          

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [16]:
checkPoint = ModelCheckpoint(r'Models/model_v2.h5',
                            monitor='val_loss',save_best_only=True,verbose=3
                            )

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkPoint,earlystop,learning_rate]

In [17]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
model.fit_generator(train_data, steps_per_epoch = train_data.samples//batch_size,
                   validation_data = validation_data,
                   validation_steps = validation_data.samples//batch_size,
                   callbacks = callbacks,
                    epochs = 5)

Epoch 1/5


C:\Users\Muhammad Talha\AppData\Local\Temp\ipykernel_1144\1074819153.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data, steps_per_epoch = train_data.samples//batch_size,


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/model/dense_1/MatMul/MatMul' defined at (most recent call last):
    File "C:\Users\Muhammad Talha\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Muhammad Talha\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Muhammad Talha\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Muhammad Talha\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\Muhammad Talha\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\Muhammad Talha\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Muhammad Talha\anacond